In [1]:
import pandas as pd
import plotly.express as px

In [2]:
covid_data = pd.read_excel('../Labb_1_Covid-19/Data/Folkhalsomyndigheten_Covid19.xlsx', sheet_name="Totalt antal per åldersgrupp")
vaccin_data = pd.read_excel('../Labb_1_Covid-19/Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx', sheet_name="Dos 1 till 3 per åldersgrupp")

# Ta bort rader som ej kommer användas samt filtrerar vaccin_data så att bara den totala datan från Sverige kommer med (istället för varje individuell region)
covid_data = covid_data.loc[(covid_data['Åldersgrupp'] != "Ålder_0_9") &
                            (covid_data['Åldersgrupp'] != "Ålder_10_19") & 
                            (covid_data['Åldersgrupp'] != "Ålder_20_29") &
                            (covid_data['Åldersgrupp'] != "Uppgift saknas")]

vaccin_data = vaccin_data.loc[(vaccin_data['Åldersgrupp'] != "12-15") &
                            (vaccin_data['Åldersgrupp'] != "16-17") & 
                            (vaccin_data['Åldersgrupp'] != "18-29") &
                            (vaccin_data['Åldersgrupp'] != "Totalt 18+") &
                            (vaccin_data['Region'] == "| Sverige |")]

# Mappa alla åldrar
ålders_grupp_mapping = {
    'Ålder_30_39': '30-39',
    'Ålder_40_49': '40-49',
    'Ålder_50_59': '50-59',
    'Ålder_60_69': '60-69',
    'Ålder_70_79': '70-79',
    'Ålder_80_89': '80-89',
    'Ålder_90_plus': '90 eller äldre'
}

# Byt namn på kolumner och celler (åldrarna)
covid_data.rename(columns={"Totalt_antal_fall": "Totalt antal fall"}, inplace=True)
covid_data["Åldersgrupp"] = covid_data['Åldersgrupp'].replace(ålders_grupp_mapping)

# Summera antal vaccinerade för varje åldersgrupp
vaccin_data = vaccin_data.groupby('Åldersgrupp')['Antal vaccinerade'].sum().reset_index()

# Sortera datan
covid_data_sorterad = covid_data.sort_values(by='Åldersgrupp').reset_index(drop=True)
vaccin_data_sorterad = vaccin_data.sort_values(by='Åldersgrupp').reset_index(drop=True)

# Skapa ett nytt dataset som kombinerar båda
kombinerad_data = pd.DataFrame({
    'Åldersgrupp': covid_data_sorterad['Åldersgrupp'],
    'Totalt antal fall': covid_data_sorterad['Totalt antal fall'],
    'Antal vaccinerade': vaccin_data_sorterad['Antal vaccinerade']
})

# Smält den kombinerade datan för att få en lämplig struktur för plotting
smält_data = kombinerad_data.melt(id_vars='Åldersgrupp', var_name='Typ', value_name='Antal')   # Kod från ChatGPT

# Plotta ett stapeldiagram
fig = px.bar(
    smält_data,
    x="Åldersgrupp",
    y="Antal",
    color="Typ",
    barmode='group',
    title='Antal fall och Antal vaccinerade för varje åldersgrupp från 30 år till 90 eller äldre',
    labels={"Typ": "Data"},
    template="simple_white"
)

fig.show()


In [3]:
covid_data = pd.read_excel('../Labb_1_Covid-19/Data/Folkhalsomyndigheten_Covid19.xlsx', sheet_name="Veckodata Region")
vaccin_data = pd.read_excel('../Labb_1_Covid-19/Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx', sheet_name="Vaccinerade tidsserie")

covid_data.rename(columns={"år": "År", "veckonummer": "Vecka", "Antal_fall_vecka": "Antal fall per vecka"}, inplace=True)

vaccin_data = vaccin_data.loc[(vaccin_data['Region'] != "| Sverige |") &
                              (vaccin_data['År'] != 2020) &
                              (vaccin_data['Vaccinationsstatus'] != "Minst 2 doser")]
covid_data = covid_data.loc[(covid_data['År'] != 2020)]

covid_data["Region"] = covid_data["Region"].replace({'Sörmland': "Södermanland", 'Jämtland Härjedalen': "Jämtland"})

vaccin_data_sorterad = vaccin_data.sort_values(['År', 'Vecka', 'Region']).reset_index(drop=True)
covid_data_sorterad = covid_data.sort_values(['År', 'Vecka', 'Region']).reset_index(drop=True)

vaccin_data_sorterad['Vaccinerade per vecka'] = vaccin_data_sorterad.groupby(['Region'])['Antal vaccinerade'].diff().fillna(0)  # Kod från ChatGPT

kombinerad_data = pd.DataFrame({
    'År': vaccin_data_sorterad['År'],
    'Vecka': vaccin_data_sorterad['Vecka'],
    'Region': vaccin_data_sorterad['Region'],
    'Vaccinerade per vecka': vaccin_data_sorterad['Vaccinerade per vecka'],
    'Antal fall per vecka': covid_data_sorterad['Antal fall per vecka'],
})

fig = px.scatter(kombinerad_data, 
                 x='Antal fall per vecka', 
                 y='Vaccinerade per vecka', 
                 color='Region', 
                 labels={'Vaccinerade per vecka': "Antal vaccinerade",
                         'Antal fall per vecka': "Antal fall"
                         },
                 hover_data=['År', 'Vecka'],
                 title='Antal covid-19 fall och vaccinationer per vecka och region för perioden 2021 - 2022',
                 template='simple_white')

fig.show()